In [1]:
import numpy as np
import pandas as pd

In [4]:
selected_columns = ['visitor_location_country_id','prop_country_id', 'prop_starrating', \
                    'prop_review_score', 'prop_location_score1', 'prop_location_score2', \
                    'prop_log_historical_price', 'position', 'price_usd','promotion_flag', \
                    'srch_query_affinity_score', 'random_bool', 'click_bool', 'booking_bool', \
                    'cr1','cr2']



# All attributes we want select
selected_columns = ['visitor_location_country_id','prop_country_id', 'prop_starrating', \
                    'prop_review_score', 'prop_location_score1', 'prop_location_score2', \
                    'prop_log_historical_price', 'price_usd','promotion_flag', \
                    'srch_query_affinity_score', 'random_bool']

# The attributes on which we are going to standardize on
# -> So for any datapoint x, we want to standardize x with respect to all datapoints with same
# 'visitor_location_country_id' AND 'prop_country_id' as x
standardize_on = ['visitor_location_country_id', 'prop_country_id']

# Of the selected attributes, the attributes which we standardize
columns_to_standardize = ['prop_starrating','price_usd', 'prop_review_score', \
                          'prop_location_score1', 'prop_location_score2', 'prop_log_historical_price']

In [5]:
# Read whole dataset
df = pd.read_csv('pcadf.csv', sep=',')

In [6]:
df.shape

(4958347, 33)

In [7]:
df_selection = df[selected_columns]

In [30]:
df_selection.median()

visitor_location_country_id    219.0000
prop_country_id                219.0000
prop_starrating                  3.0000
prop_review_score                4.0000
prop_location_score1             2.7700
prop_location_score2             0.0690
prop_log_historical_price        4.9100
price_usd                      122.0000
promotion_flag                   0.0000
srch_query_affinity_score      -20.4513
random_bool                      0.0000
dtype: float64

In [31]:
df_selection.mean()

visitor_location_country_id    175.340453
prop_country_id                173.973897
prop_starrating                  3.180525
prop_review_score                3.777777
prop_location_score1             2.872589
prop_location_score2             0.130385
prop_log_historical_price        4.317913
price_usd                      254.209590
promotion_flag                   0.215620
srch_query_affinity_score      -24.146418
random_bool                      0.295900
dtype: float64

In [35]:
df_selection = df_selection.fillna(df_selection.median())

In [163]:
# For each "key" ('visitor_location_country_id', 'prop_country_id'), we
# standardize a numeric attribute as a-> (a-m)/s, m=mean,s=std
# The dict "transf_param" remembers for each key encountered in whole training set the mean and std
# as tuple (m,s) where m,s both numpy arrays

transf_param = {}
k = 0
for name,group in df_selection[standardize_on+columns_to_standardize].groupby(by=standardize_on):
    mean = np.array(group[columns_to_standardize].mean())
    #print("mean is: " + str(mean))
    std = np.array(group[columns_to_standardize].std())
    #print("std is: " + str(std))

    if True in np.isnan(mean):# or True in np.isnan(std):
        print("mean" + str(name))
    
    if True in np.isnan(std):
        std = np.ones(len(columns_to_standardize))
        print("std " + str(name))
#        print(group)

    # If std gives 0, replace with 1
    epsilon = 0.01            # Reason is that we have found some cases where all values are 0.69, but std wasnt 0
    std[std<epsilon] = 1
    
    transf_param[name] = (mean,std)
    k=k+1
    if k % 18000 == 0:
        print(k)
        break

std (23, 165)
std (36, 99)
std (77, 119)
std (79, 202)
std (99, 208)
std (103, 163)
std (110, 165)
std (214, 202)
std (219, 208)


In [164]:
#print(len(df_selection['srch_destination_id'].unique()))
print(len(df_selection['visitor_location_country_id'].unique()))
print(len(df_selection['prop_country_id'].unique()))

210
172


In [165]:
len(transf_param)

4058

In [166]:
# Save the dict as file.
np.save('transf_param.npy', transf_param)

In [167]:
# After computing all means and std, and saved on file, we can normalize the test set.

In [168]:
# Load dict from file
saved_transf_param = np.load('transf_param.npy').item()

In [169]:
# Load whole test data
df_test = pd.read_csv('test_set_lines_500k.csv', sep=',')
#df_test = pd.read_csv('test_set_VU_DM_2014.csv', sep=',')

In [170]:
df_test.shape

(499999, 50)

In [171]:
df_test = df_test[selected_columns]

In [172]:
df_test.describe()

,visitor_location_country_id,prop_country_id,prop_starrating,prop_review_score,prop_location_score1,prop_location_score2,prop_log_historical_price,price_usd,promotion_flag,srch_query_affinity_score,random_bool
count,499999.000000,499999.000000,499999.000000,499245.000000,499999.000000,392126.000000,499999.000000,4.999990e+05,499999.000000,31395.000000,499999.000000
mean,175.334581,173.233276,3.184944,3.780674,2.881491,0.130132,4.327942,2.928037e+02,0.219198,-24.614399,0.297205
std,65.783272,68.669938,1.050451,1.044247,1.533979,0.160393,1.829020,2.520525e+04,0.413704,16.774284,0.457028
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000e-02,0.000000,-241.027700,0.000000
25%,100.000000,100.000000,3.000000,3.500000,1.790000,0.018400,4.450000,8.500000e+01,0.000000,-30.893400,0.000000
50%,219.000000,219.000000,3.000000,4.000000,2.830000,0.068000,4.920000,1.230000e+02,0.000000,-20.507000,0.000000
75%,219.000000,219.000000,4.000000,4.500000,4.060000,0.179900,5.320000,1.856900e+02,0.000000,-13.767900,1.000000
max,230.000000,230.000000,5.000000,5.000000,6.980000,0.999600,6.210000,9.661340e+06,1.000000,-3.155400,1.000000


In [173]:
df_test.count()

visitor_location_country_id    499999
prop_country_id                499999
prop_starrating                499999
prop_review_score              499245
prop_location_score1           499999
prop_location_score2           392126
prop_log_historical_price      499999
price_usd                      499999
promotion_flag                 499999
srch_query_affinity_score       31395
random_bool                    499999
dtype: int64

In [174]:
# Have to fill in missing values too...
df_test = df_test.fillna(df_selection.median())

In [175]:
df_test.count()

visitor_location_country_id    499999
prop_country_id                499999
prop_starrating                499999
prop_review_score              499999
prop_location_score1           499999
prop_location_score2           499999
prop_log_historical_price      499999
price_usd                      499999
promotion_flag                 499999
srch_query_affinity_score      499999
random_bool                    499999
dtype: int64

In [176]:
#print(len(df_test['srch_destination_id'].unique()))
print(len(df_test['visitor_location_country_id'].unique()))
print(len(df_test['prop_country_id'].unique()))

143
142


In [177]:
df_selection[(df_selection['visitor_location_country_id'] == 2) & (df_selection['prop_country_id'] == 32)].std()

visitor_location_country_id     0.000000
prop_country_id                 0.000000
prop_starrating                 0.480384
prop_review_score               1.244217
prop_location_score1            1.685590
prop_location_score2            0.072103
prop_log_historical_price       0.000000
price_usd                      77.783724
promotion_flag                  0.518875
srch_query_affinity_score       0.000000
random_bool                     0.000000
dtype: float64

In [178]:
num_attr_to_stand = len(columns_to_standardize)
avg_mean = np.zeros(num_attr_to_stand)
avg_std = np.zeros(num_attr_to_stand)

for key,value in saved_transf_param.items():

    avg_mean = avg_mean + value[0]
    avg_std = avg_std + value[1]

avg_mean = avg_mean / len(saved_transf_param)
avg_std = avg_std / len(saved_transf_param)
print(avg_mean)
print(avg_std)

[3.61359367e+00 4.29132391e+02 3.44736381e+00 3.04779461e+00
 1.17998774e-01 4.40103201e+00]
[8.97319190e-01 1.08478456e+03 1.27501350e+00 1.09386185e+00
 1.20702599e-01 1.08035115e+00]


In [179]:
# In the test set, we go through each ('visitor_location_country_id', 'prop_country_id') group
# and standardize the values of (some of ) the attributes

# Remember howmany groups in total, and howmany of them were not encountered before
num_data_points = 0
num_data_points_unknown = 0
num_unknowns = 0
k = 0
for name,group in df_test.groupby(by=standardize_on):
    if k % 6000 == 0:
        print(k)
    k = k+1
    num_data_points = num_data_points + group.shape[0]
    
    if name in saved_transf_param:
        # Means we have encountered the key in training set
        cur_params = saved_transf_param[name]
#        print(cur_params)
    else:
        # Means we havent encountered the key in training set
        print("Unknown group " + str(k) + ", with key: " + str(name))
        num_unknowns = num_unknowns + 1
        num_data_points_unknown = num_data_points_unknown + group.shape[0]
        # For unknown key, we have use:
        #  - mean = avg mean of all known keys
        #  - std = avg std of all known keys
        cur_params = (avg_mean*np.ones(num_attr_to_stand),avg_std*np.ones(num_attr_to_stand))

#    print(df_test.loc[(df_test['visitor_location_country_id'] == name[0]) & (df_test['srch_destination_id'] == name[1])])
    df_test.loc[(df_test['visitor_location_country_id'] == name[0]) & (df_test['prop_country_id'] == name[1]), \
                columns_to_standardize] = \
            (df_test.loc[(df_test['visitor_location_country_id'] == name[0]) & \
                        (df_test['prop_country_id'] == name[1]), columns_to_standardize] - cur_params[0]) / cur_params[1]
#    print(df_test.loc[(df_test['visitor_location_country_id'] == name[0]) & (df_test['srch_destination_id'] == name[1])])

print("total num of unknowns is: " + str(num_unknowns))
print("total num groups is: " + str(k))
print("total num of unknown data points is: " + str(num_data_points_unknown))
print("total num of data points is: " + str(num_data_points))

0
Unknown group 3, with key: (2, 56)
Unknown group 7, with key: (2, 221)
Unknown group 8, with key: (3, 48)
Unknown group 24, with key: (5, 48)
Unknown group 28, with key: (5, 220)
Unknown group 37, with key: (10, 202)
Unknown group 38, with key: (11, 127)
Unknown group 56, with key: (13, 163)
Unknown group 66, with key: (14, 158)
Unknown group 69, with key: (14, 211)
Unknown group 73, with key: (15, 26)
Unknown group 77, with key: (15, 48)
Unknown group 90, with key: (16, 32)
Unknown group 92, with key: (16, 129)
Unknown group 95, with key: (18, 9)
Unknown group 100, with key: (22, 18)
Unknown group 102, with key: (22, 68)
Unknown group 105, with key: (23, 4)
Unknown group 118, with key: (23, 201)
Unknown group 122, with key: (23, 217)
Unknown group 125, with key: (26, 4)
Unknown group 131, with key: (28, 50)
Unknown group 134, with key: (30, 192)
Unknown group 157, with key: (31, 83)
Unknown group 191, with key: (32, 63)
Unknown group 201, with key: (32, 222)
Unknown group 214, with 

In [180]:
df_test.describe()

,visitor_location_country_id,prop_country_id,prop_starrating,prop_review_score,prop_location_score1,prop_location_score2,prop_log_historical_price,price_usd,promotion_flag,srch_query_affinity_score,random_bool
count,499999.000000,499999.000000,499999.000000,499999.000000,499999.000000,499999.000000,499999.000000,4.999990e+05,499999.000000,499999.000000,499999.000000
mean,175.334581,173.233276,-0.001527,-0.002882,0.004015,0.009114,-0.020173,9.371014e+01,0.219198,-20.712702,0.297205
std,65.783272,68.669938,1.007711,1.013278,1.109021,1.084909,1.190061,2.497270e+04,0.413704,4.322853,0.457028
min,1.000000,1.000000,-10.206207,-16.782928,-60.074071,-4.792055,-30.664914,-3.160000e+02,0.000000,-241.027700,0.000000
25%,100.000000,100.000000,-0.710445,-0.451397,-0.680050,-0.635620,0.069328,-7.945599e-02,0.000000,-20.451300,0.000000
50%,219.000000,219.000000,0.064770,0.151539,0.039591,-0.306247,0.330861,-3.081505e-02,0.000000,-20.451300,0.000000
75%,219.000000,219.000000,0.765414,0.754064,0.756748,0.172508,0.548485,1.438829e-02,0.000000,-20.451300,1.000000
max,230.000000,230.000000,3.171904,3.227486,50.133618,48.308228,6.210000,9.661340e+06,1.000000,-3.155400,1.000000


In [181]:
df_test.count()

visitor_location_country_id    499999
prop_country_id                499999
prop_starrating                499999
prop_review_score              499999
prop_location_score1           499999
prop_location_score2           499999
prop_log_historical_price      499999
price_usd                      499999
promotion_flag                 499999
srch_query_affinity_score      499999
random_bool                    499999
dtype: int64

In [182]:
df_test[df_test['price_usd'] > 1000]

,visitor_location_country_id,prop_country_id,prop_starrating,prop_review_score,prop_location_score1,prop_location_score2,prop_log_historical_price,price_usd,promotion_flag,srch_query_affinity_score,random_bool
489415,202,100,1.450706,1.239309,6.197749,0.179605,4.073617,7.719411e+06,0,-20.4513,0
489416,202,100,1.450706,0.765005,6.180059,0.179605,2.561759,7.043117e+06,0,-20.4513,0
489417,202,100,0.267235,0.290702,6.180059,0.179605,1.469862,3.276644e+06,1,-20.4513,0
489418,202,100,2.634176,0.765005,6.126989,41.381062,4.703558,9.661340e+06,0,-20.4513,0
489419,202,100,0.267235,0.765005,6.126989,40.299786,1.931818,3.076364e+06,0,-20.4513,0
489420,202,100,0.267235,0.290702,5.702431,2.907003,2.729743,4.279672e+06,1,-20.4513,0
489421,202,100,1.450706,0.765005,6.180059,0.179605,3.149704,6.753277e+06,0,-20.4513,0
489422,202,100,2.634176,0.765005,6.126989,47.271595,4.073617,5.016168e+06,1,-20.4513,0


In [185]:
saved_transf_param[(202,100)]
#saved_transf_param[(219,100)]

(array([2.77419355, 0.0116129 , 3.69354839, 1.14645161, 0.06677419,
        4.82      ]),
 array([0.84497248, 1.        , 1.05417756, 0.56529372, 0.01239277,
        0.23811762]))

In [192]:
(df_test[(df_test['visitor_location_country_id'] == 202) & \
             (df_test['prop_country_id'] == 100)][columns_to_standardize] )#- saved_transf_param[(202,100)][0])\
#/saved_transf_param[(202,100)][1]

,prop_starrating,price_usd,prop_review_score,prop_location_score1,prop_location_score2,prop_log_historical_price
489415,1.450706,7.719411e+06,1.239309,6.197749,0.179605,4.073617
489416,1.450706,7.043117e+06,0.765005,6.180059,0.179605,2.561759
489417,0.267235,3.276644e+06,0.290702,6.180059,0.179605,1.469862
489418,2.634176,9.661340e+06,0.765005,6.126989,41.381062,4.703558
489419,0.267235,3.076364e+06,0.765005,6.126989,40.299786,1.931818
489420,0.267235,4.279672e+06,0.290702,5.702431,2.907003,2.729743
489421,1.450706,6.753277e+06,0.765005,6.180059,0.179605,3.149704
489422,2.634176,5.016168e+06,0.765005,6.126989,47.271595,4.073617


In [184]:
df_selection[(df_selection['visitor_location_country_id'] == 219) & \
             (df_selection['prop_country_id'] == 100) & \
             (df_selection['price_usd']>10000)]

,visitor_location_country_id,prop_country_id,prop_starrating,prop_review_score,prop_location_score1,prop_location_score2,prop_log_historical_price,price_usd,promotion_flag,srch_query_affinity_score,random_bool
4115920,219,100,2,4.0,0.69,0.0022,4.77,10736.90,0,-20.4513,0
4115921,219,100,4,4.0,4.19,0.0275,5.28,15248.24,0,-20.4513,0
4115923,219,100,3,4.5,4.51,0.0972,5.38,17593.82,1,-20.4513,0
4115924,219,100,2,3.5,1.79,0.0121,4.72,18252.54,1,-20.4513,0
4115925,219,100,4,3.5,2.08,0.0132,4.95,16564.70,1,-20.4513,0
4115927,219,100,3,4.5,4.49,0.0498,5.33,19458.56,0,-20.4513,0
4115928,219,100,2,4.0,1.39,0.0058,4.70,12991.59,1,-20.4513,0


In [193]:
# Save standardized attributres to file
df_test.to_csv('test_stand.csv')